In [ ]:
import jupedsim as jps
from shapely import Polygon
import pathlib

In [ ]:
area = Polygon([(-0.5, 0), (41, 0), (41, 2), (-0.5, 2)])

In [ ]:
spawning_area = Polygon([(-0.5, 0), (0.2, 0), (0.2, 2), (-0.5, 2)])
num_agents = 1
positions = jps.distributions.distribute_by_number(
    polygon=spawning_area,
    number_of_agents=num_agents,
    distance_to_agents=0.4,
    distance_to_polygon=0.2,
    seed=1,
)
exit_area = Polygon([(40, 0), (41, 0), (41, 2), (40, 2)])

In [ ]:
trajectory_file = "Test1.sqlite"  # output file
simulation = jps.Simulation(
    model=jps.CollisionFreeSpeedModel(),
    geometry=area,
    trajectory_writer=jps.SqliteTrajectoryWriter(
        output_file=pathlib.Path(trajectory_file)
    ),
)

exit_id = simulation.add_exit_stage(exit_area)
journey = jps.JourneyDescription([exit_id])
journey_id = simulation.add_journey(journey)

In [ ]:
ids = []
for pos in positions:
    id = simulation.add_agent(
        jps.CollisionFreeSpeedModelAgentParameters(
            journey_id=journey_id, stage_id=exit_id, position=pos, v0=0.001
        )
    )
    ids.append(id)

In [ ]:
premovement_time = 10
dt = simulation.delta_time()
premovement_iterations = int(premovement_time / dt)
# Run Simulation for given number of iterations
simulation.iterate(premovement_iterations)
# Agent may start now
simulation.agent(ids[0]).model.v0 = 1.33
# Run simulation to the end
while simulation.agent_count() > 0:
    simulation.iterate()

print(f"Simulation time: {simulation.iteration_count()*dt} seconds.")    

In [ ]:
from jupedsim.internal.notebook_utils import animate, read_sqlite_file

trajectory_data, walkable_area = read_sqlite_file(trajectory_file)
animate(trajectory_data, walkable_area, every_nth_frame=5)